This notebook is based on https://github.com/dmpierre/air-ccs/blob/main/sage/air-to-ccs.ipynb. This notebook implements
- Plonk-to-CCS conversion algorithm, found in the [CCS paper](https://eprint.iacr.org/2023/552), Section 2.2.
- A PoC of incorporating Halo2 lookup into CCS+.
[The explanation of math behind the algorithm is here](https://hackmd.io/@pnyda/SkG0qaUuA)


# Derive a CCS instance from a custom gate and constant columns with no respect to copy constraints and lookups

In [1]:
# Classes to represent a custom gate

p = 41
F = GF(p)
R.<x> = PolynomialRing(F)
# The modulus of the field doesn't really matter.
# Here I use one from https://youtu.be/A3edAQDPnDY?si=K5E4EP3tTwyrKY5P&t=2544

from dataclasses import dataclass
from typing import Literal

@dataclass(eq=True, frozen=True, order=True)
class Column:
    index: int
    kind: Literal["advice", "constant"]

@dataclass(eq=True, frozen=True, order=True)
class RelativeCellPointer:
    column: Column
    row_offset: int

@dataclass(eq=True, frozen=True, order=True)
class Monomial:
    coefficient: F
    variables: list[RelativeCellPointer]

@dataclass(eq=True, frozen=True, order=True)
class CustomGate:
    monomials: list[Monomial]

    def cell_pointers(self) -> list[RelativeCellPointer]:
        # We sometimes refer to the same set of cells from multiple monomials.
        # In that case a naive implementation would generate redundant M_j.
        # To avoid it we take distinct set of RelativeCellPointers.
        # and assign ID j for M_j by sorting it
        return sorted({variable for monomial in self.monomials for variable in monomial.variables})

    def t(self) -> list[int]:
        return len(self.cell_pointers())

    def c(self) -> list[int]:
        return [monomial.coefficient for monomial in self.monomials]

    def s(self) -> list[set[int]]:
        return [{j := self.cell_pointers().index(variable) for variable in monomial.variables} for monomial in self.monomials]
        
    def m(self, constant_columns: list[list[F]]):
        table_height = len(constant_columns[0])
        matrices = []
    
        for cell_pointer in self.cell_pointers():
            # Here I add 1 because 1 is always in Z
            m_j = matrix(F, table_height, self.num_advice_columns() * table_height + 1)
            for y in range(table_height):
                if cell_pointer.column.kind == "advice":
                    x = 1 + self.l() + cell_pointer.column.index * table_height + (y + cell_pointer.row_offset) % table_height
                    m_j[y, x] = 1
                elif cell_pointer.column.kind == "constant":
                    # We assume here constant columns are followed by advice columns. This is a PoC code and it's not essential.
                    constant_column_index = cell_pointer.column.index - self.num_advice_columns()
                    # 0 here means that we multiply the rhs by the first element of Z which is always 1
                    m_j[y, 0] = constant_columns[constant_column_index][(y + cell_pointer.row_offset) % table_height]
                else:
                    pass

            matrices.append(m_j)
    
        return matrices

    def l(self) -> int:
        # Number of public i/o
        # To make this PoC code simple we assume l=0
        return 0

    def num_advice_columns(self) -> int:
        return len({cell_pointer.column.index for cell_pointer in self.cell_pointers() if cell_pointer.column.kind == "advice"})

# Check if a CCS instance is satisfied

In [2]:
# https://github.com/dmpierre/air-ccs/blob/main/sage/air-to-ccs.ipynb

def ccs_is_satisfied(F, z, matrices, multisets, constants):
    satisfied_instance_witness = vector(F, [0 for i in range(matrices[0].dimensions()[0])])
    z_final = vector(F, [0 for i in range(matrices[0].dimensions()[0])])
    for i, c in enumerate(constants):
        multiset = multisets[i]
        z_i = vector(F, [1 for i in range(matrices[0].dimensions()[0])])
        for j in multiset:
            z_i = z_i.pairwise_product(matrices[j] * z)
        z_final += c * z_i
    return z_final == satisfied_instance_witness

# Original Plonk

In [3]:
# g = q_m * a * b + q_l * a + q_r * b + q_o * c + q_c

## advice columns
a = Column(0, "advice")
b = Column(1, "advice")
c = Column(2, "advice")

## constant columns
q_m = Column(3, "constant")
q_l = Column(4, "constant")
q_r = Column(5, "constant")
q_o = Column(6, "constant")
q_c = Column(7, "constant")

a_cur = RelativeCellPointer(a, 0)
b_cur = RelativeCellPointer(b, 0)
c_cur = RelativeCellPointer(c, 0)
q_m_cur = RelativeCellPointer(q_m, 0)
q_l_cur = RelativeCellPointer(q_l, 0)
q_r_cur = RelativeCellPointer(q_r, 0)
q_o_cur = RelativeCellPointer(q_o, 0)
q_c_cur = RelativeCellPointer(q_c, 0)

g_plonk = CustomGate([
    Monomial(F(1), [q_m_cur, a_cur, b_cur]),
    Monomial(F(1), [q_l_cur, a_cur]),
    Monomial(F(1), [q_r_cur, b_cur]),
    Monomial(F(1), [q_o_cur, c_cur]),
    Monomial(F(1), [q_c_cur]),
])

print("c_plonk", g_plonk.c())
print("s_plonk", g_plonk.s())

# Example values taken from https://hackmd.io/nQtquuk9QCGiB9EhUPgXvg#CCS-and-Plonkish-intro
q_m_values = [F(1), F(1), F(0), F(0)]
q_l_values = [F(0), F(0), F(2), F(0)]
q_r_values = [F(0), F(0), F(2), F(0)]
q_o_values = [F(-1), F(-1), F(-1), F(0)]
q_c_values = [F(0), F(0), F(0), F(0)]
constant_columns = [q_m_values, q_l_values, q_r_values, q_o_values, q_c_values]
m_plonk = g_plonk.m(constant_columns)
print("m_plonk", *m_plonk, sep="\n\n")

a_values = [F(0), F(1), F(1), F(1)]
b_values = [F(0), F(1), F(2), F(2)]
c_values = [F(0), F(1), F(6), F(3)]

z_plonk = vector(F, [F(1)] + a_values + b_values + c_values)
print("z_plonk", z_plonk)
assert ccs_is_satisfied(F, z_plonk, m_plonk, g_plonk.s(), g_plonk.c())

c_plonk [1, 1, 1, 1, 1]
s_plonk [{0, 1, 3}, {0, 4}, {1, 5}, {2, 6}, {7}]
m_plonk

[0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0]

[0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0]

[0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1]

[1 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]

[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]

[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]

[40  0  0  0  0  0  0  0  0  0  0  0  0]
[40  0  0  0  0  0  0  0  0  0  0  0  0]
[40  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0]

[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 

# Fibonacci

In [4]:
# enable(X) * (fib(X) + fib(Xω) - fib(Xω^2)) = 0
fib = Column(0, "advice")
enable = Column(1, "constant")

fib_cur = RelativeCellPointer(fib, 0)
fib_next = RelativeCellPointer(fib, 1)
fib_next_next = RelativeCellPointer(fib, 2)
enable_cur = RelativeCellPointer(enable, 0)

g_fib = CustomGate([
    Monomial(1, [enable_cur, fib_cur]),
    Monomial(1, [enable_cur, fib_next]),
    Monomial(-1, [enable_cur, fib_next_next]),
])
print("c_fib", g_fib.c())
print("s_fib", g_fib.s())

enable_values = [F(1), F(1), F(0), F(0)]
m_fib = g_fib.m([enable_values])
print("m_fib", *m_fib, sep="\n\n")

fib_values = [F(1), F(1), F(2), F(3)]
z_fib = vector(F, [F(1)] + fib_values)
assert ccs_is_satisfied(F, z_fib, m_fib, g_fib.s(), g_fib.c())

c_fib [1, 1, -1]
s_fib [{0, 3}, {1, 3}, {2, 3}]
m_fib

[0 1 0 0 0]
[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]

[0 0 1 0 0]
[0 0 0 1 0]
[0 0 0 0 1]
[0 1 0 0 0]

[0 0 0 1 0]
[0 0 0 0 1]
[0 1 0 0 0]
[0 0 1 0 0]

[1 0 0 0 0]
[1 0 0 0 0]
[0 0 0 0 0]
[0 0 0 0 0]


# Apply copy constraints to a CCS instance

In [5]:
from dataclasses import dataclass

@dataclass
class AbsoluteCellPointer:
    column: Column
    row_index: int

def z_index(cell: AbsoluteCellPointer, table_height: int) -> int: 
    return 1 + cell.column.index * table_height + cell.row_index

# Step 1:
# If M_j refers to multiple elements of Z that have to be the same,
# update the reference in such a way that it all refers to the same element.

# We also generate deduplication_map whose key is the original z_index and the value is z_index into which the original was deduplicated
# This is needed to add lookup constraints afterwards

def apply_copy_constraints(copy_constraints: list[list[AbsoluteCellPointer]], matrices: list[matrix], constant_columns: list[list[F]]) -> (list[matrix], list[int]):
    table_height = len(constant_columns[0])
    num_advice_columns = (matrices[0].ncols() - 1) // table_height

    deduplication_map = [i for i in range(matrices[0].ncols())]
    matrices = [copy(m_j) for m_j in matrices]
    for m_j in matrices:
        for row_index in range(m_j.nrows()):
            for equal_cells in copy_constraints:
                for cell in equal_cells[1:]:
                    if m_j[row_index, z_index(cell, table_height)] != 0:
                        # Deduplicate equal_cells[1:] into equal_cells[0]
                        if equal_cells[0].column.kind == "advice":
                            m_j[row_index, z_index(equal_cells[0], table_height)] = m_j[row_index, z_index(cell, table_height)]
                            m_j[row_index, z_index(cell, table_height)] = 0
                            deduplication_map[z_index(cell, table_height)] = z_index(equal_cells[0], table_height)
                        elif equal_cells[0].column.kind == "constant":
                            # We assume here constant columns are followed by advice columns. This is a PoC code and it's not essential.
                            constant_column_index = equal_cells[0].column.index - num_advice_columns
                            # 0 here means that we multiply the rhs by the first element of Z which is always 1
                            m_j[row_index, 0] = constant_columns[constant_column_index][equal_cells[0].row_index]
                            m_j[row_index, z_index(cell, table_height)] = 0
                        else:
                            pass

    return matrices, deduplication_map


# Step 2:
# Remove elements of Z that is referenced by no M_j
def reduce_witness_size(old_matrices: list[matrix], old_z: vector, dedup_map: list[int]) -> (list[matrix], vector):
    used_z_indices = {x for m_j in old_matrices for y in range(m_j.nrows()) for x in range(m_j.ncols()) if m_j[y, x] != 0}
    unused_z_indices = set(range(len(old_z))) - used_z_indices
    print("unused_z_indices", unused_z_indices)
    print("used_z_indices", used_z_indices)

    # Update Z
    new_z = list(old_z)
    for unused_z_index in reversed(sorted(list(unused_z_indices))):
        new_z.pop(unused_z_index)

    new_matrices = []
    for (j, old_m_j) in enumerate(old_matrices):
        new_m_j = matrix(F, old_m_j.nrows(), len(used_z_indices))
        for y in range(old_m_j.nrows()):
            for x in range(old_m_j.ncols()):
                offset = len([unused_z_index for unused_z_index in unused_z_indices if unused_z_index < x])
                if old_m_j[y, x] != 0:
                    # Update M_j
                    new_m_j[y, x - offset] = old_m_j[y, x]

                    # Update deduplication map
                    for original_z_index in range(len(dedup_map)):
                        if x == dedup_map[original_z_index]:
                            dedup_map[original_z_index] -= offset
        
        new_matrices.append(new_m_j)

    return (new_matrices, vector(F, new_z))

# Apply copy constraints to m_plonk

In [6]:
copy_constraints = [
    [AbsoluteCellPointer(a, 0), AbsoluteCellPointer(b, 0), AbsoluteCellPointer(c, 0)],
    [AbsoluteCellPointer(a, 1), AbsoluteCellPointer(b, 1), AbsoluteCellPointer(c, 1)],
    [AbsoluteCellPointer(a, 3), AbsoluteCellPointer(b, 3), AbsoluteCellPointer(c, 3)],
    [AbsoluteCellPointer(q_l, 2), AbsoluteCellPointer(b, 2)],  # This copy constraint was not in https://hackmd.io/nQtquuk9QCGiB9EhUPgXvg
]
(deduplicated_m, dedup_map) = apply_copy_constraints(copy_constraints, m_plonk, constant_columns)
(cleaned_up_m, cleaned_up_z) = reduce_witness_size(deduplicated_m, z_plonk, dedup_map)
print("original", *m_plonk, sep="\n\n")
print("deduplicated", *deduplicated_m, sep="\n\n")
print("cleaned_up_m", *cleaned_up_m, sep="\n\n")
print("original_z", z_plonk)
print("cleaned_up_z", cleaned_up_z)
assert ccs_is_satisfied(F, cleaned_up_z, cleaned_up_m, g_plonk.s(), g_plonk.c())

unused_z_indices {5, 6, 7, 8, 9, 10, 12}
used_z_indices {0, 1, 2, 3, 4, 11}
original

[0 1 0 0 0 0 0 0 0 0 0 0 0]
[0 0 1 0 0 0 0 0 0 0 0 0 0]
[0 0 0 1 0 0 0 0 0 0 0 0 0]
[0 0 0 0 1 0 0 0 0 0 0 0 0]

[0 0 0 0 0 1 0 0 0 0 0 0 0]
[0 0 0 0 0 0 1 0 0 0 0 0 0]
[0 0 0 0 0 0 0 1 0 0 0 0 0]
[0 0 0 0 0 0 0 0 1 0 0 0 0]

[0 0 0 0 0 0 0 0 0 1 0 0 0]
[0 0 0 0 0 0 0 0 0 0 1 0 0]
[0 0 0 0 0 0 0 0 0 0 0 1 0]
[0 0 0 0 0 0 0 0 0 0 0 0 1]

[1 0 0 0 0 0 0 0 0 0 0 0 0]
[1 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]

[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]

[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[2 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]

[40  0  0  0  0  0  0  0  0  0  0  0  0]
[40  0  0  0  0  0  0  0  0  0  0  0  0]
[40  0  0  0  0  0  0  0  0  0  0  0  0]
[ 0  0  0  0  0  0  0  0  0  0  0  0  0]

[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 

# ZeroTest
or GateCheck? I don't know the industry standard way to call it

In [7]:
# The generator of 4th roots of unity
omega = F(9)
subgroup_order = 4
assert omega^subgroup_order == F(1)

z_in_zerotest = R.one()
for i in range(4):
    z_in_zerotest *= (x - omega^i)

def prove_zero_test(f):
    q = f // z_in_zerotest
    return q

def verify_zero_test(f, q) -> bool:
    challenge = F.random_element()
    return f(challenge) == q(challenge) * z_in_zerotest(challenge)

# Grand Product Argument

In [8]:
# The generator of 4th roots of unity
omega = F(9)
assert omega^4 == F(1)

# We want to succinctly check that seq1 is a reordered version of seq2
seq1 = vector(F, [0, 1, 2, 3])
seq2 = vector(F, [3, 2, 1, 0])

f = R.one()
for a_i in seq1:
    f = f * (x + a_i)

g = R.one()
for b_i in seq2:
    g = g * (x + b_i)

gamma = F.random_element()
assert f(gamma) == g(gamma)


def interpolate_at_roots_of_unity(seq):
    points = []
    for (i, elem) in enumerate(seq):
        points.append((omega^i, elem))

    return R.lagrange_polynomial(points)


beta = F.random_element()
gamma = F.random_element()

# Constrain a_prime to be reordered a
# Constrain s_prime to be reordered s
# With just one grand product argument, by using two random challenges
def prove_grand_product(a_prime, s_prime, a, s):
    z_trace = [F(1)]

    for i in range(subgroup_order):
        z_next = z_trace[-1]
        z_next *= (a(omega^i) + beta) / (a_prime(omega^i) + beta)
        z_next *= (s(omega^i) + gamma) / (s_prime(omega^i) + gamma)
        z_trace.append(z_next)

    # When generating the trace column Z, we have to do division
    # Sometimes we encounter a situation where we divide 0, because we shift the dividened by a random challenge
    # In that case this code panics with ZeroDivisionError
    # How do provers in production avoid such cases?

    z_trace[0] = z_trace.pop()

    z = interpolate_at_roots_of_unity(z_trace)
    constraint1 = z(omega * x) * (a_prime(x) + beta) * (s_prime(x) + gamma) - z(x) * (a(x) + beta) * (s(x) + gamma)
    q1 = prove_zero_test(constraint1)
    
    l0 = R.lagrange_polynomial([(omega^i, 1 if i == 0 else 0) for i in range(subgroup_order)])
    constraint2 = l0(x) * (F.one() - z(x))
    q2 = prove_zero_test(constraint2)
    
    return z, q1, q2

def verify_grand_product(a_prime, s_prime, a, s, z, q1, q2) -> bool:
    constraint1 = z(omega * x) * (a_prime(x) + beta) * (s_prime(x) + gamma) - z(x) * (a(x) + beta) * (s(x) + gamma)
    l0 = R.lagrange_polynomial([(omega^i, 1 if i == 0 else 0) for i in range(subgroup_order)])
    constraint2 = l0(x) * (F.one() - z(x))
    return verify_zero_test(constraint1, q1) and verify_zero_test(constraint2, q2)

a = interpolate_at_roots_of_unity([3, 2, 1, 0])
a_prime = interpolate_at_roots_of_unity([0, 1, 2, 3])
s = interpolate_at_roots_of_unity([27, 24, 40, 8])
s_prime = interpolate_at_roots_of_unity([8, 24, 27, 40])
proof = prove_grand_product(a_prime, s_prime, a, s)
assert verify_grand_product(a_prime, s_prime, a, s, *proof)

# "Copy this or that" argument

In [9]:
# Constrains a_prime to be s_prime but with some elements duplicated, some elements missing
def prove_copy_this_or_that(a_prime, s_prime):
    constraint1 = (a_prime(x) - s_prime(x)) * (a_prime(x) - a_prime(x/omega))
    l0 = R.lagrange_polynomial([(omega^i, 1 if i == 0 else 0) for i in range(subgroup_order)])
    constraint2 = l0 * (a_prime(x) - s_prime(x))
    return prove_zero_test(constraint1), prove_zero_test(constraint2)

def verify_copy_this_or_that(a_prime, s_prime, q1, q2) -> bool:
    constraint1 = (a_prime(x) - s_prime(x)) * (a_prime(x) - a_prime(x/omega))
    l0 = R.lagrange_polynomial([(omega^i, 1 if i == 0 else 0) for i in range(subgroup_order)])
    constraint2 = l0 * (a_prime(x) - s_prime(x))
    return verify_zero_test(constraint1, q1) and verify_zero_test(constraint2, q2)
    

s_prime = interpolate_at_roots_of_unity([0, 1, 2, 3])
a_prime = interpolate_at_roots_of_unity([0, 0, 2, 3])

q1, q2 = prove_copy_this_or_that(a_prime, s_prime)
assert verify_copy_this_or_that(a_prime, s_prime, q1, q2)

# Halo2 lookup

In [10]:
def sort_s(a_trace, s_trace) -> list[F]:
    a_sorted = sorted(a_trace)
    s_sorted = []

    # We're gonna pop an element from either of these 2
    s_not_in_a = [s_i for s_i in s_trace if s_i not in a_trace]
    s_in_a = [s_i for s_i in s_trace if s_i in a_trace]
    
    for (i, a_i) in enumerate(a_sorted):
        if a_i in s_in_a:  # Have we encountered this a_i before?
            # If a_i is new value we haven't seen before
            # Copy a_i to s_i
            s_i = s_in_a.pop(s_in_a.index(a_i))
        else:
            # If a_i is duplicated
            # Copy some random unused element to s_i
            s_i = s_not_in_a.pop()

        s_sorted.append(s_i)

    assert len(s_not_in_a) == 0
    assert len(s_in_a) == 0
    print("a_prime", a_sorted)
    print("s_prime", s_sorted)
    
    return s_sorted
    

def prove_halo2_lookup(a_trace, s_trace) -> bool:
    a = interpolate_at_roots_of_unity(a_trace)
    s = interpolate_at_roots_of_unity(s_trace)
    a_prime = interpolate_at_roots_of_unity(sorted(a_trace))
    s_prime = interpolate_at_roots_of_unity(sort_s(a_trace, s_trace))
    
    z_in_grand_product, q1, q2 = prove_grand_product(a_prime, s_prime, a, s)
    q3, q4 = prove_copy_this_or_that(a_prime, s_prime)
    return z_in_grand_product, q1, q2, q3, q4


def verify_halo2_lookup(a_trace, s_trace, z_in_grand_product, q1, q2, q3, q4):
    a = interpolate_at_roots_of_unity(a_trace)
    s = interpolate_at_roots_of_unity(s_trace)
    a_prime = interpolate_at_roots_of_unity(sorted(a_trace))
    s_prime = interpolate_at_roots_of_unity(sort_s(a_trace, s_trace))
    
    return verify_grand_product(a_prime, s_prime, a, s, z_in_grand_product, q1, q2) and verify_copy_this_or_that(a_prime, s_prime, q3, q4)


# Valid
lookup_table = vector(F, [7, 24, 40, 8])
must_be_in_table = vector(F, [24, 8, 40, 24])
proof = prove_halo2_lookup(must_be_in_table, lookup_table)
assert verify_halo2_lookup(must_be_in_table, lookup_table, *proof)

# Invalid
lookup_table = vector(F, [7, 24, 40, 8])
must_be_in_table = vector(F, [24, 9, 40, 24])
proof = prove_halo2_lookup(must_be_in_table, lookup_table)
assert not verify_halo2_lookup(must_be_in_table, lookup_table, *proof)

a_prime [8, 24, 24, 40]
s_prime [8, 24, 7, 40]
a_prime [8, 24, 24, 40]
s_prime [8, 24, 7, 40]
a_prime [9, 24, 24, 40]
s_prime [8, 24, 7, 40]
a_prime [9, 24, 24, 40]
s_prime [8, 24, 7, 40]


# CCS+

In [11]:
print(dedup_map)

column_to_lookup = c_cur.column
L = [dedup_map[z_index(AbsoluteCellPointer(column_to_lookup, row_index), subgroup_order)] for row_index in range(subgroup_order)]
print("L", L)
Z_o = [cleaned_up_z[o] for o in L]
print("Z_o", Z_o)

T = [6, 3, 0, 1]
proof = prove_halo2_lookup(Z_o, T)
assert verify_halo2_lookup(Z_o, T, *proof)

[0, 1, 2, 3, 4, 1, 2, 7, 4, 1, 2, 5, 4]
L [1, 2, 5, 4]
Z_o [0, 1, 6, 1]
a_prime [0, 1, 1, 6]
s_prime [0, 1, 3, 6]
a_prime [0, 1, 1, 6]
s_prime [0, 1, 3, 6]


# Realizations
In this code I first generate M_j with no respect to copy constraints nor lookups, and then deduplicate it. This feels ugly and redundant. Maybe I should beforehand generate a map that keeps track of which cell in the table goes to which element of Z, and then generate M_j. That way lookup implementation would look cleaner as it becomes easy to tell which element of Z has to be in the lookup table. 

This code only supports one custom gate, but to support multiple custom gates, we can just increase the height of M_j to constrain multiple gates to evalutate to 0. That way we don't have to sample the random challenge gamma described in [Halo2 book](https://zcash.github.io/halo2/design/proving-system/vanishing.html) that was necessary to combine multiple custom gates into one.